In [1]:
import sys; sys.path.append("../")

import pandas as pd

from lib.match import (
    ColumnsIndex, JaroWinklerSimilarity, StringSimilarity, DateSimilarity, ThresholdMatcher
)
from lib.date import combine_date_columns
from clean.new_orleans_harbor_pd_pprr import clean_personnel_2008, clean_personnel_2020
from match.new_orleans_harbor_pd import prepare_post_data, match_uid_with_cprr
from lib.columns import rearrange_personnel_columns, rearrange_personnel_history_columns

In [2]:
df20 = clean_personnel_2020()
df20

/Users/khoipham/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4438: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
../lib/clean.py:166: FutureWarning: The default value of regex will change from True to False in a future version.
  return series.str.strip().str.replace(r"[^\w-]+", " ").str.replace(r"\s+", " ")\
/Users/khoipham/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1842: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value
../lib/clean.py:156: FutureWarnin

,first_name,last_name,rank_desc,hourly_salary,middle_initial,term_year,term_month,term_day,hire_year,hire_month,hire_day,pay_effective_year,pay_effective_month,pay_effective_day,uid,agency,data_production_year
0,casey,adams,police sergeant--port,17.00,m,2010,2,28,1980,1,31,1997,4,14,0e62eddff885ed636a82cd726b390596,New Orleans Harbor PD,2020
1,casey,adams,police sergeant--port,17.68,m,2010,2,28,1980,1,31,2003,12,10,0e62eddff885ed636a82cd726b390596,New Orleans Harbor PD,2020
2,casey,adams,police sergeant--port,18.39,m,2010,2,28,1980,1,31,2004,8,10,0e62eddff885ed636a82cd726b390596,New Orleans Harbor PD,2020
3,casey,adams,police sergeant--port,19.13,m,2010,2,28,1980,1,31,2005,8,10,0e62eddff885ed636a82cd726b390596,New Orleans Harbor PD,2020
4,casey,adams,police sergeant-a,19.13,m,2010,2,28,1980,1,31,2005,8,29,0e62eddff885ed636a82cd726b390596,New Orleans Harbor PD,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
910,brianne,waters,police officer 3-a,21.60,m,2020,5,11,2001,3,5,2015,10,1,b1014901a4b7ddee9ecd880405127d81,New Orleans Harbor PD,2020
911,brianne,waters,police sergeant-a,24.34,m,2020,5,11,2001,3,5,2018,5,14,b1014901a4b7ddee9ecd880405127d81,New Orleans Harbor PD,2020
912,brianne,waters,police sergeant-a,25.07,m,2020,5,11,2001,3,5,2018,7,15,b1014901a4b7ddee9ecd880405127d81,New Orleans Harbor PD,2020
913,brianne,waters,police sergeant-a,25.07,m,2020,5,11,2001,3,5,2018,7,15,b1014901a4b7ddee9ecd880405127d81,New Orleans Harbor PD,2020


In [3]:
df08 = clean_personnel_2008()
df08

../lib/clean.py:166: FutureWarning: The default value of regex will change from True to False in a future version.
  return series.str.strip().str.replace(r"[^\w-]+", " ").str.replace(r"\s+", " ")\
../lib/clean.py:156: FutureWarning: The default value of regex will change from True to False in a future version.
  return series.str.strip().str.replace(r"[^\d\.]", "").astype("float64")


,first_name,middle_initial,last_name,rank_desc,hourly_salary,hire_year,hire_month,hire_day,term_year,term_month,term_day,pay_effective_year,pay_effective_month,pay_effective_day,agency,data_production_year,uid
0,gregg,j,wanciak,police officer 2--port,6.63,1990,11,9,1991,3,29,1991,3,29,New Orleans Harbor PD,2008,f4840b946dd5a85d51e5005995f31924
1,david,c,forchia,police officer 2--port,9.74,1980,3,13,1991,4,26,1991,4,26,New Orleans Harbor PD,2008,52dc0a6aae3cbedb6ed08b62df2cd3db
2,ronald,j,scully jr,police officer 2--port,6.89,1990,4,14,1991,5,9,1991,5,9,New Orleans Harbor PD,2008,f100f89da914eaaa8d1e92d73dfd2da4
3,albert,j,perry,police lieutenant--port,12.33,1973,8,30,1991,9,12,1991,9,12,New Orleans Harbor PD,2008,0e51be31402e56750127160ed6f914e9
4,james,b,bingham,police officer 2--port,7.98,1989,1,30,1991,9,23,1991,9,23,New Orleans Harbor PD,2008,74fbc9c91b06481b5b2ca88bdcc32a6f
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164,robert,w,lincoln,police officer 3-a,15.89,1997,8,4,2008,12,15,2006,2,4,New Orleans Harbor PD,2008,9f8d86f6e844de7b7371123cc6c7b704
165,robert,w,lincoln,police officer 3-a,16.53,1997,8,4,2008,12,15,2007,2,4,New Orleans Harbor PD,2008,9f8d86f6e844de7b7371123cc6c7b704
166,robert,w,lincoln,police officer 3-a,17.25,1997,8,4,2008,12,15,2007,7,1,New Orleans Harbor PD,2008,9f8d86f6e844de7b7371123cc6c7b704
167,robert,w,lincoln,police officer 3-a,18.98,1997,8,4,2008,12,15,2007,11,1,New Orleans Harbor PD,2008,9f8d86f6e844de7b7371123cc6c7b704


In [6]:
cprr = pd.read_csv("../data/clean/cprr_new_orleans_harbor_pd_2020.csv")
matcher = match_uid_with_cprr(cprr, df20)

In [7]:
matcher.get_sample_pairs()

first_name  \
score_range pair_idx sim_score row_key                                       
1.00-0.95   0        1.000000  c550f567113a27df7d07ea462fb1a5a5      bryan   
                               e6278fa24d4f3a03c7b1fc41f07dd6de      bryan   
            1        1.000000  e0afe1e6553097708bcd87c772cc4141     howard   
                               2e448f6aba55cd31091991480f1af231     howard   
            2        0.969254  9d9e4d8548e120eee8f80b9a459e7b5a      lamus   
                               cac9967e392f4edf4da41c4e478f96b5      lamus   

                                                                last_name fc  
score_range pair_idx sim_score row_key                                        
1.00-0.95   0        1.000000  c550f567113a27df7d07ea462fb1a5a5     arena  b  
                               e6278fa24d4f3a03c7b1fc41f07dd6de     arena  b  
            1        1.000000  e0afe1e6553097708bcd87c772cc4141  dennison  h  
                               2e448f6aba55cd31091991480f1af231  dennison  h  
            2        0.969254  9d9e4d8548e120eee8f80b9a459e7b5a     lundy  l  
                               cac9967e392f4edf4da41c4e478f96b5  lundy jr  l